## Step 1: Imports

In [1]:
import sys
sys.path.append('../phraseextraction/')

import utility
import ranking
import pandas as pd
from rule import grammar
import candidate_generation
from ranking import TextRank


## Settings
pd.set_option('display.max_rows', 500)

/home/dsp/miniconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to /home/dsp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/dsp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Step 2: Reading Data

In [2]:
with open('../dataset/AMZN_0001018724_10K_20191231_Item1_excerpt.txt') as company_description:
    description = company_description.read()
#print(description)

## Step 3: Candidate Generation: Grammar Rules Based

In [3]:
grammar_model = candidate_generation.Grammar_Keyphrase(grammar)
key_phrases = grammar_model.get_keyphrases(description)
print(key_phrases)

['herein by reference contain forward-looking', 'statements based on expectations', 'expressed in forward-looking statements', 'principal corporate offices are located', 'common stock is listed', 'similar terms include', 's most customer-centric company', 'competitor focus , passion for invention', 'customer sets , consisting', 'consumers , sellers , developers', 'services , such as advertising', 'sellers , vendors , publishers', 'sponsored ads , display', 'segments reflect the way', 'business performance and manages', 'net sales is contained', 'results of Whole Foods', 'online and physical stores', 'focus on selection , price', 'hundreds of millions of unique', 'third parties across dozens', 'websites , mobile apps', 'sell electronic devices', 'produce media content', 'customers low prices', 'delivery , easy-to-use functionality', 'includes unlimited free shipping', 'unlimited streaming of tens', 'movies and TV episodes , including', 'fulfill customer orders', 'number of ways , includ

## Step 4: Post Processing: Cleaning candidate phrases

### Remove Punctuations and Digits

In [4]:
#1. Remove punnctuations in each phrase
phrases_wo_puncts = [utility.remove_punct_num(ph) for ph in key_phrases]
print(phrases_wo_puncts)

['herein by reference contain forward-looking', 'statements based on expectations', 'expressed in forward-looking statements', 'principal corporate offices are located', 'common stock is listed', 'similar terms include', 's most customer-centric company', 'competitor focus passion for invention', 'customer sets consisting', 'consumers sellers developers', 'services such as advertising', 'sellers vendors publishers', 'sponsored ads display', 'segments reflect the way', 'business performance and manages', 'net sales is contained', 'results of Whole Foods', 'online and physical stores', 'focus on selection price', 'hundreds of millions of unique', 'third parties across dozens', 'websites mobile apps', 'sell electronic devices', 'produce media content', 'customers low prices', 'delivery easy-to-use functionality', 'includes unlimited free shipping', 'unlimited streaming of tens', 'movies and TV episodes including', 'fulfill customer orders', 'number of ways including', 'fulfillment and del

### Remove Stopwords

In [5]:
#2. Remove stop words from phrases

custom_stop_words = ['include','including','provide','provides','offers','related','new', 'allowed','help','overall','array','without','with','persuant',
                 'legacy','existing','exists','addition','additional','ensure','used','uses','process','offered','described','using','provided','sold',
                 'allows','providing','following','follows','rate','labels','come','vast','covers','end','late','use','allowing','exiting','full','many',
                'includes','earlier','become','upon','allow','among','required','based','higher','exciting','seen','certain','see','continue','various',
                'trademarks','patents','copyrights','intellectual','patent','trade-secret','trademark','copyright']

phrases_wo_stopwords = [utility.remove_stopwords(ph, custom_stop_words) for ph in phrases_wo_puncts]
print(phrases_wo_stopwords)

['reference forward-looking', 'statements expectations', 'expressed forward-looking statements', 'principal corporate offices located', 'common stock listed', 'similar terms', 'customer-centric company', 'competitor focus passion invention', 'customer sets consisting', 'consumers sellers developers', 'services advertising', 'sellers vendors publishers', 'sponsored ads display', 'segments reflect', 'business performance manages', 'net sales contained', 'results Whole Foods', 'online physical stores', 'focus selection price', 'hundreds millions unique', 'parties dozens', 'websites mobile apps', 'sell electronic devices', 'produce media content', 'customers low prices', 'delivery easy-to-use functionality', 'unlimited free shipping', 'unlimited streaming tens', 'movies TV episodes', 'fulfill customer orders', 'number ways', 'fulfillment delivery networks', 'arrangements countries', 'operate customer service centers', 'supplemented co-sourced arrangements', 'percentage sales per-unit', 'ac

### Remove Non english words

In [6]:
#3. Remove non english words in each phrase
phrases_wo_nonenglish_words = [utility.remove_non_english(phrase) for phrase in phrases_wo_stopwords]
print(phrases_wo_nonenglish_words)

['reference forward-looking', 'statements expectations', 'expressed forward-looking statements', 'principal corporate offices located', 'common stock listed', 'similar terms', 'company', 'competitor focus passion invention', 'customer sets consisting', 'consumers sellers developers', 'services advertising', 'sellers vendors publishers', 'sponsored ads display', 'segments reflect', 'business performance manages', 'net sales contained', 'results Whole Foods', 'online physical stores', 'focus selection price', 'hundreds millions unique', 'parties dozens', 'websites mobile apps', 'sell electronic devices', 'produce media content', 'customers low prices', 'delivery easy-to-use functionality', 'unlimited free shipping', 'unlimited streaming tens', 'movies TV episodes', 'fulfill customer orders', 'number ways', 'fulfillment delivery networks', 'arrangements countries', 'operate customer service centers', 'supplemented co-sourced arrangements', 'percentage sales per-unit', 'activity fees', 'se

### Only keep Trigrams & Quadgrams

In [7]:
#4. Only keeping bigrams, trigrams, quadgrams
final_phrases = candidate_generation.get_ngrams(phrases_wo_nonenglish_words, (3,4))
print(final_phrases)

['expressed forward-looking statements', 'principal corporate offices located', 'common stock listed', 'competitor focus passion invention', 'customer sets consisting', 'consumers sellers developers', 'sellers vendors publishers', 'sponsored ads display', 'business performance manages', 'net sales contained', 'results Whole Foods', 'online physical stores', 'focus selection price', 'hundreds millions unique', 'websites mobile apps', 'sell electronic devices', 'produce media content', 'customers low prices', 'delivery easy-to-use functionality', 'unlimited free shipping', 'unlimited streaming tens', 'movies TV episodes', 'fulfill customer orders', 'fulfillment delivery networks', 'operate customer service centers', 'supplemented co-sourced arrangements', 'percentage sales per-unit', 'serve developers enterprises', 'set global compute', 'authors independent publishers', 'authors publishers choose', 'filmmakers skill app developers', 'large variety product types', 'retailers publishers ve

## Step 5: TextRank


### Score based on Word Embeddings Method

In [8]:
TR_WordEmbedding= TextRank(description, method= "WordEmbeddings")
WE_Phrases = TR_WordEmbedding.rank_phrases(final_phrases)

In [9]:
display(WE_Phrases)

,Phrases,Score
0,information technology services products,0.017447
1,operate customer service centers,0.016707
2,change business practices,0.016691
3,supply direct consumers,0.016655
4,large variety product types,0.016563
5,development business models,0.016344
6,potential customers restrictive,0.016341
7,customers greater brand recognition,0.016308
8,seller enterprise services,0.016046
9,produce media content,0.015837


### Score based on Window Size Method

In [10]:
TR_WindowSize = TextRank(original_text = description, method= "WindowSize")
WS_Phrases = TR_WindowSize.rank_phrases(final_phrases)

In [11]:
display(WS_Phrases)

,Phrases,Score
0,operate customer service centers,0.067889
1,customers low prices,0.051408
2,fulfill customer orders,0.050850
3,change business practices,0.050236
4,sell electronic devices,0.047606
5,produce media content,0.040315
6,restrictive distribution agreements,0.039685
7,numerous financial leadership roles,0.038699
8,corporate governance information,0.036529
9,international domain names,0.035637
